In [1]:
! pip install pyedgar


In [2]:
from pyedgar import Filing
from pyedgar import EDGARIndex
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [ ]:
file_type = '8-K'

In [20]:
filesave_directory = 'C:/Users/Cheng Jiang/Desktop/Research/Project with Ron/Lawyer CEO and Innovation - with Barbara/Projects/Project 20210210 8K files download/'

In [3]:
# set True for first time downloading and False for future time
all_indices = EDGARIndex(force_download=True)

108it [01:48,  1.01s/it]                     
Done downloading, extracting...
100%|██████████| 6/6 [03:34<00:00, 35.70s/it] 
Done!


In [67]:
database = all_indices['8-K']

In [68]:
print(database)

             cik                                        name   form  \
0             20                    K TRON INTERNATIONAL INC    8-K   
1             20                    K TRON INTERNATIONAL INC    8-K   
2             20                    K TRON INTERNATIONAL INC    8-K   
3             20                    K TRON INTERNATIONAL INC    8-K   
4             20                    K TRON INTERNATIONAL INC  8-K/A   
...          ...                                         ...    ...   
1767663  1837019         Ford Credit Auto Lease Trust 2021-A    8-K   
1767664  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767665  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767666  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767667  1838040  Toyota Auto Receivables 2021-A Owner Trust    8-K   

          filedate             accession  
0       1995-07-10  0000893220-95-000441  
1       2001-10-17  0000893220-01-500767  
2       2002-06-26

In [69]:
# only get files later than 1995
database = database.loc[(database['filedate'] > '1995-01-01')]   

In [70]:
# create the new variable called EDGARURL, which has a format like https://www.sec.gov/Archives/edgar/data/1750/0001104659-17-062143.txt
database['EDGARURL'] = 'https://www.sec.gov/Archives/edgar/data/' + database['cik'].apply(str) + '/' + database['accession'].apply(str) +'.txt'

In [71]:
print(database)

             cik                                        name   form  \
0             20                    K TRON INTERNATIONAL INC    8-K   
1             20                    K TRON INTERNATIONAL INC    8-K   
2             20                    K TRON INTERNATIONAL INC    8-K   
3             20                    K TRON INTERNATIONAL INC    8-K   
4             20                    K TRON INTERNATIONAL INC  8-K/A   
...          ...                                         ...    ...   
1767663  1837019         Ford Credit Auto Lease Trust 2021-A    8-K   
1767664  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767665  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767666  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767667  1838040  Toyota Auto Receivables 2021-A Owner Trust    8-K   

          filedate             accession  \
0       1995-07-10  0000893220-95-000441   
1       2001-10-17  0000893220-01-500767   
2       2002-06

In [72]:
total_entries = database['EDGARURL'].count()
print(total_entries)     # there are totally 1,767,668 8K files

1767668


### use the first item for experiment 

In [73]:
cik = database.iloc[1767667, 0]   # the CIK is in the first row and first column 
print(cik)

1838040


In [74]:
accession = database.iloc[1767667, 4]   # the accession is in the first row and 5th column 
print(accession)

0000929638-21-000206


In [75]:
url = database.iloc[1767667, 5]  # the accession is in the first row and 6th column 
print(url)

https://www.sec.gov/Archives/edgar/data/1838040/0000929638-21-000206.txt


In [76]:
r = requests.get(url)
raw = r.text
print(raw)

<SEC-DOCUMENT>0000929638-21-000206.txt : 20210204
<SEC-HEADER>0000929638-21-000206.hdr.sgml : 20210204
<ACCEPTANCE-DATETIME>20210204170035
ACCESSION NUMBER:		0000929638-21-000206
CONFORMED SUBMISSION TYPE:	8-K
PUBLIC DOCUMENT COUNT:		10
CONFORMED PERIOD OF REPORT:	20210202
<DEPOSITOR-CIK>0001131131
<SPONSOR-CIK>0000834071
ITEM INFORMATION:		Entry into a Material Definitive Agreement
ITEM INFORMATION:		Financial Statements and Exhibits
FILED AS OF DATE:		20210204
DATE AS OF CHANGE:		20210204
ABS ASSET CLASS:             	Auto loans

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			TOYOTA AUTO FINANCE RECEIVABLES LLC
		CENTRAL INDEX KEY:			0001131131
		STANDARD INDUSTRIAL CLASSIFICATION:	ASSET-BACKED SECURITIES [6189]
		IRS NUMBER:				334836519
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			0331

	FILING VALUES:
		FORM TYPE:		8-K
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	333-228027
		FILM NUMBER:		21591988

	BUSINESS ADDRESS:	
		STREET 1:		6565 HEADQUARTERS DRIVE
		STREET 2:		ATTEN

Step 5: specify the beginning point and ending point for text info extraction

In [77]:
docs_start = re.compile(r'<DOCUMENT>')
docs_end = re.compile(r'</DOCUMENT>')
docs_type = re.compile(r'<TYPE>[^\n]+')

print(docs_start)
print(docs_end)
print(docs_type)

re.compile('<DOCUMENT>')
re.compile('</DOCUMENT>')
re.compile('<TYPE>[^\\n]+')


In [78]:
doc_s = [x.end() for x in docs_start.finditer(raw)]
doc_e = [x.start() for x in docs_end.finditer(raw)]
doc_t = [x[len('<TYPE>'):] for x in docs_type.findall(raw)]

print(doc_s)
print(doc_e)
print(doc_t)

[1938, 34420, 214523, 535273, 1230018, 1953784, 2094269, 2224216, 2334002, 2533708]
[34398, 214501, 535251, 1229996, 1953762, 2094247, 2224194, 2333980, 2533686, 2539341]
['8-K', 'EX-1.1', 'EX-4.1', 'EX-4.2', 'EX-4.3', 'EX-4.4', 'EX-4.5', 'EX-4.6', 'EX-4.7', 'EX-36.1']


In [79]:
filetype = doc_t[0]
print(filetype)

8-K


In [80]:
document = {}
document[filetype] = raw[doc_start:doc_end]
print(document)

{'8-K': '\n<TYPE>8-K\n<SEQUENCE>1\n<FILENAME>form8k.htm\n<TEXT>\n<html>\n  <head>\n    <title></title>\n    <!-- Licensed to: mlb12\n         Document created using EDGARfilings PROfile 7.3.1.0\n         Copyright 1995 - 2021 Broadridge -->\n  </head>\n<body style="font-family: \'Times New Roman\', Times, serif; font-size: 10pt; text-align: left; color: #000000;" bgcolor="#ffffff">\n  <div style="text-align: center; font-family: \'Times New Roman\'; font-weight: bold;">UNITED STATES</div>\n  <div style="text-align: center; font-family: \'Times New Roman\'; font-weight: bold;">SECURITIES AND EXCHANGE COMMISSION</div>\n  <div style="text-align: center; font-family: \'Times New Roman\'; font-weight: bold;">WASHINGTON, DC&#160; 20549</div>\n  <div>&#160;</div>\n  <div style="text-align: center; font-family: \'Times New Roman\'; font-weight: bold;">FORM 8-K</div>\n  <div>&#160;</div>\n  <div style="text-align: center; font-family: \'Times New Roman\'; font-weight: bold;">CURRENT REPORT PURS

In [81]:
print(document[filetype])


<TYPE>8-K
<SEQUENCE>1
<FILENAME>form8k.htm
<TEXT>
<html>
  <head>
    <title></title>
    <!-- Licensed to: mlb12
         Document created using EDGARfilings PROfile 7.3.1.0
         Copyright 1995 - 2021 Broadridge -->
  </head>
<body style="font-family: 'Times New Roman', Times, serif; font-size: 10pt; text-align: left; color: #000000;" bgcolor="#ffffff">
  <div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">UNITED STATES</div>
  <div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">SECURITIES AND EXCHANGE COMMISSION</div>
  <div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">WASHINGTON, DC&#160; 20549</div>
  <div>&#160;</div>
  <div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">FORM 8-K</div>
  <div>&#160;</div>
  <div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">CURRENT REPORT PURSUANT TO</div>
  <div style="text-align: 

In [49]:
print(filetype, " : ", url)

8-K  :  https://www.sec.gov/Archives/edgar/data/1838040/0000929638-21-000206.txt


In [63]:
item_xcontent = BeautifulSoup(document[filetype], 'lxml')
print(item_xcontent)

<html><body><type>8-K
<sequence>1
<filename>form8k.htm
<text>
<title></title>
<!-- Licensed to: mlb12
         Document created using EDGARfilings PROfile 7.3.1.0
         Copyright 1995 - 2021 Broadridge -->
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">UNITED STATES</div>
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">SECURITIES AND EXCHANGE COMMISSION</div>
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">WASHINGTON, DC  20549</div>
<div> </div>
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">FORM 8-K</div>
<div> </div>
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">CURRENT REPORT PURSUANT TO</div>
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">SECTION 13 OR 15(d) OF THE</div>
<div style="text-align: center; font-family: 'Times New Roman'; font-weight: bold;">SECURIT

In [64]:
content = item_xcontent.get_text()
print(content)

8-K
1
form8k.htm



UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
WASHINGTON, DC  20549
 
FORM 8-K
 
CURRENT REPORT PURSUANT TO
SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 



Date of report (Date of earliest event reported):  February 2, 2021



 




TOYOTA AUTO RECEIVABLES 2021-A OWNER TRUST





(Exact name of Issuing Entity as specified in its charter)



 




TOYOTA AUTO FINANCE RECEIVABLES LLC





(Exact name of Depositor/Registrant as specified in its charter)



 




TOYOTA MOTOR CREDIT CORPORATION





(Exact name of Sponsor as specified in its charter)









Delaware





(State or Other Jurisdiction of Incorporation)











333-228027
333-228027-09


 


95-4836519
95-3775816






(Commission File Number)

 

(IRS Employer Identification No.)











6565 Headquarters Drive, W2-3D, Plano, Texas


 


75024-5965





(Address of Principal Executive Offices)

 

(Zip Code)








Registrant’s telephone number, including area code: (469) 486-9

In [50]:
filename = (filesave_directory + str(cik) +'/'+ str(accession) + '.txt')
print(filename)

C:/Users/Cheng Jiang/Desktop/Research/Project with Ron/Lawyer CEO and Innovation - with Barbara/Projects/Project 20210210 8K files download/1838040/0000929638-21-000206.txt


In [65]:
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, "w", encoding='utf-8') as f:
    f.write(str(content))